# Tiền xử lý dữ liệu

## Mục tiêu
- Đưa ra góc nhìn tổng quan về dữ liệu
- Chuyển dữ liệu ngày tháng sang kiểu `datetime`
- Phân tách dữ liệu ra thành các tệp nhỏ hơn

## Khai báo thư viện cần dùng

In [1]:
import pandas as pd
from pathlib import Path

## Đọc/Ghi dữ liệu

- Dữ liệu đầu vào/đầu ra được lưu trữ trong thư mục `data`
- Dữ liệu được lấy tại [đây](https://www.kaggle.com/datasets/georgesaavedra/covid19-dataset)
- Dữ liệu được đăng lên bởi **Mathieu, E., Ritchie, H., Ortiz-Ospina, E. et al.**

In [2]:
SRC_LINK = Path('data/')
covid_df = pd.read_csv(SRC_LINK / 'covid-data.csv')

### Tổng quan về dữ liệu

In [3]:
covid_df.shape

(166326, 67)

Dữ liệu bao gồm 166326 theo dõi, tương ứng với 67 cột (trường dữ liệu)

In [4]:
# In ra thông tin của DataFrame
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166326 entries, 0 to 166325
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    166326 non-null  object 
 1   continent                                   156370 non-null  object 
 2   location                                    166326 non-null  object 
 3   date                                        166326 non-null  object 
 4   total_cases                                 163293 non-null  float64
 5   new_cases                                   163133 non-null  float64
 6   new_cases_smoothed                          161150 non-null  float64
 7   total_deaths                                145451 non-null  float64
 8   new_deaths                                  145487 non-null  float64
 9   new_deaths_smoothed                         143390 non-null  float64
 

> Vì mục tiêu của nhóm là sẽ xây dựng mô hình dự đoán số ca nhiễm nên sẽ sử dụng thông tin từ 6 cột sau
- `iso_code`: International unique identifier of the country
- `continent`: Continent where the country is located
- `location`: Name of the country.
- `date`: Date when the recording was done.
- `total_cases`: Since here the features are explicitly explained by their name.
- `new_cases`: The new cases is recorded

In [5]:
# Lấy ra 6 cột cần dùng
covid_df = covid_df[['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases']]
# In 5 dòng đầu của dữ liệu
covid_df.head()

,iso_code,continent,location,date,total_cases,new_cases
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0


In [6]:
# Kiểm tra các giá trị bị thiếu trong `covid_df`
covid_df.isnull().sum()


iso_code          0
continent      9956
location          0
date              0
total_cases    3033
new_cases      3193
dtype: int64

> Dữ liệu bị thiếu bao gồm ở các cột `continent` chiếm khoảng gần 6%, `total_cases` chiếm khoảng gần 2% và `new_cases` chiếm khoảng gần 2% trên tổng số theo dõi


> **Nhận xét**: Số dữ liệu bị thiếu (missing data) chiếm không quá nhiều, tuy nhiên việc có nên bỏ đi không nhóm sẽ trình bày trong mục sau (ở file data_analyst.ipynb)

In [7]:
# Dữ liệu đang xét là dữ liệu về chuỗi thời gian nên chúng ta cần chuyển dữ liệu ngày tháng từ dạng string về dạng datetime
covid_df['date'] = pd.to_datetime(covid_df['date'])

In [8]:
covid_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166326 entries, 0 to 166325
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   iso_code     166326 non-null  object        
 1   continent    156370 non-null  object        
 2   location     166326 non-null  object        
 3   date         166326 non-null  datetime64[ns]
 4   total_cases  163293 non-null  float64       
 5   new_cases    163133 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 7.6+ MB


In [9]:
'''
    Như kết quả đầu ra, chúng ta thấy rằng dữ liệu được thu từ ngày 01-01-2020 
    và theo dõi cuối cùng được ghi nhận là vào ngày 05-03-2022

'''
date_range = str(covid_df['date'].dt.date.min()) + ' to ' +str(covid_df['date'].dt.date.max())
print(date_range)

2020-01-01 to 2022-03-05


### Phân chia dữ liệu thành những tệp nhỏ hơn

Trong mục này, nhóm sẽ phân chia dữ liệu theo `continent`

In [10]:
'''
    Vì cột continent có những giá trị NaN vậy nên chúng ta sẽ cần lưu ra thành một tệp riêng
    để tìm hiểu xem những giá trị NaN đó có nghĩa là gì
'''
nan_data = covid_df[covid_df['continent'].isnull()]
print('nan-covid-data.csv\n', nan_data.shape)
nan_data.to_csv(SRC_LINK / 'nan-covid-data.csv', index=False)
# Tìm giá trị duy nhất trong cột continent
continents = covid_df['continent'].unique()
print(list(continents))
for c in continents:
    '''
        - Sử dụng lower() để chuyển string sang dạng chữ bé hơn
        - Sử dụng strip() để đảm bảo rằng ở đầu và cuối của string không có khoảng trắng
        - Cuối cùng, thay thế các khoảng trắng thành -
    '''
    continent = str(c).lower().strip().replace(' ', '-')
    if continent != 'nan':
        filename = continent + '-data.csv'
        print(filename)
        print(covid_df.query(f"continent == '{c}'").shape)
        covid_df.query(f"continent == '{c}'").to_csv(SRC_LINK / filename, index=False)

nan-covid-data.csv
 (9956, 6)
['Asia', nan, 'Europe', 'Africa', 'North America', 'South America', 'Oceania']
asia-data.csv
(36200, 6)
europe-data.csv
(37088, 6)
africa-data.csv
(39417, 6)
north-america-data.csv
(24931, 6)
south-america-data.csv
(9517, 6)
oceania-data.csv
(9217, 6)


## Kết Luận

- Sau khi phân tách các dữ liệu ra thành các tập nhỏ hơn, chúng ta có tổng cộng 6 tệp csv được ghi lại vào thư mục `data`
-  Cụ thể
   -  `asia-data.csv` có 36200 theo dõi
   -  `europe-data.csv` có 37088 theo dõi
   -  `africa-data.csv` có 39417 theo dõi
   -  `north-america-data.csv` có 24931 theo dõi
   -  `south-america-data.csv` có 9517 theo dõi
   -  `oceania-data.csv` có 9217 theo dõi
   -  `nan-covid-data.csv` có 9956 theo dõi